# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. Launch the web UI (CPU Version - Clean Install)

#@markdown CPU-optimized setup for running models without GPU

import os
from pathlib import Path
import shutil

os.environ.pop('PYTHONPATH', None)
os.environ.pop('MPLBACKEND', None)

# Force clean install - remove old GPU installation
if Path('text-generation-webui').exists():
  print("\033[1;33m --> Removing old installation to force CPU-only reinstall...\033[0m")
  shutil.rmtree('text-generation-webui')

print("\033[1;32;1m\n --> Installing the web UI for CPU. This will take a while.\033[0;37;0m\n")

!git clone https://github.com/oobabooga/text-generation-webui
%cd text-generation-webui

# Install the project in an isolated environment (CPU mode)
!GPU_CHOICE=N \
LAUNCH_AFTER_INSTALL=FALSE \
INSTALL_EXTENSIONS=FALSE \
./start_linux.sh

# Parameters - CPU-friendly model
model_url = "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF" #@param {type:"string"}
branch = "main" #@param {type:"string"}
command_line_flags = "--cpu --gpu-layers 0 --ctx-size 2048 --threads 2" #@param {type:"string"}
api = True #@param {type:"boolean"}

if api:
  for param in ['--api', '--public-api']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # Download the model
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')
    if branch.strip() not in ['', 'main']:
        output_folder += f"_{branch}"
        !python download-model.py {model_url} --branch {branch}
    else:
        !python download-model.py {model_url}
else:
    output_folder = ""

# Start the web UI
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder != "":
    cmd += f" --model {output_folder}"

!$cmd
```

**Key changes:**
1. Added code to delete `text-generation-webui` folder if it exists
2. This forces a complete reinstall with `GPU_CHOICE=N` which will install CPU-only binaries
3. Changed `--threads 4` to `--threads 2` (Colab free tier typically has 2 CPU cores)

**Your CMD_FLAGS.txt stays the same:**
```
--cpu --gpu-layers 0 --ctx-size 2048 --threads 2 --api --public-api --share